# Set up environments

In [1]:
!pip install -q beautifulsoup4==4.12.2 PyMuPDFb==1.24.10 tqdm==4.66.5 duckdb==0.10.0 tabulate==0.9.0

In [2]:
import os

if not ".git" in os.listdir():
    !git clone https://github.com/batprem/pyo3-experiment
    os.chdir("pyo3-experiment")

# Download 56-1 file

In [3]:
import requests
from bs4 import BeautifulSoup
import os
import zipfile
from tqdm import tqdm


ROOT = "downloaded-56-1"
# SYMBOL = "PTT"  # Change this symbol
SYMBOL = "KBANK"  # Change this symbol
FILE_PATH = f"{ROOT}/{SYMBOL}"
PATH_TO_TARGET = f"{FILE_PATH}/{SYMBOL}.zip"
session = requests.Session()



os.makedirs(FILE_PATH, exist_ok=True)
print("Searching the balance sheet file")

response = session.get(
    f"https://market.sec.or.th/public/idisc/th/Viewmore/fs-r561?searchSymbol={SYMBOL}",
)
soup = BeautifulSoup(response.text)
url = soup.find_all("tr")[1].find_all("td")[-1].find("a")["href"]

print(f"Found at {url}")
# Sizes in bytes.
total_size = int(response.headers.get("content-length", 0))
block_size = 1024
response = session.get(url, stream=True)

print("Downloading...")
with tqdm(total=total_size, unit="B", unit_scale=True) as progress_bar:
    with open(PATH_TO_TARGET, "wb") as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)

    
# with open(PATH_TO_TARGET, "wb") as f:
#     f.write(requests.get(url).content)

with zipfile.ZipFile(PATH_TO_TARGET, 'r') as zip_ref:
    zip_ref.extractall(FILE_PATH)
print("Done")

Searching the balance sheet file
Found at https://market.sec.or.th/public/idisc/Download?FILEID=dat/f56/0016ONE120320242034060731T.zip
Downloading...


11.4MB [00:01, 6.46MB/s]                                                                                                                                                                                   

Done


# Extract table of contents

In [4]:
import fitz
import pandas as pd

def to_html(filepath: str):
    doc = fitz.open(filepath)
    for i, page in enumerate(doc):
        text = page.get_text("html")
        with open(f"pymupdf-page-{i}.html", "w") as fp:
            fp.write(text)
    doc.close()
    
doc = fitz.open(f'{FILE_PATH}/STRUCTURE{SYMBOL}T.PDF')
toc = pd.DataFrame(doc.get_toc())
toc.columns = ["hierachy", "topic", "page"]
toc = toc[toc["topic"] != "Bookmark"].reset_index(drop=True)
toc

,hierachy,topic,page
0,1,Cover,1
1,1,TableContent,2
2,1,StructureAndOperation,4
3,1,RiskManagement,8
4,1,ESG,12
5,1,CorporateGovernancePolicy,16
6,1,CorporateGovernanceStructure,18
7,1,ResultOfCorporateGovernance,31


In [28]:
bussiness_type = doc[3].get_text().split("ประเภทธุรกิจ\n:")[1].strip().split("\nเลขทะเบียนบริษัท")[0]
print(bussiness_type)

ธนาคารกสิกรไทย จำกัด (มหาชน) ประกอบกิจการธนาคารพาณิชย์ ธุรกิจ
หลักทรัพย์และธุรกิจที่เกี่ยวเนื่องตามที่ได้รับอนุญาตไว้ในพระราชบัญญัติธุรกิจ
สถาบันการเงินฯ และพระราชบัญญัติหลักทรัพย์และตลาดหลักทรัพย์ฯ และ
ประกาศที่เกี่ยวข้อง โดย ณ วันที่ 31 ธันวาคม 2566 ธนาคารมีเครือข่ายสาขา
จำนวน 809 สาขา และเครื่องอิเล็กทรอนิกส์สำหรับทำธุรกรรมด้วยตนเอง
10,903 เครื่อง ครอบคลุมทุกพื้นที่และเพียงพอต่อความต้องการของลูกค้า
นอกจากนี้ ธนาคารมีเครือข่ายการให้บริการในต่างประเทศรวม 16 แห่ง ใน 8
ประเทศ มีศูนย์กลางการดำเนินงานและให้บริการที่สำนักงานใหญ่


In [5]:
first_page_index = toc.index[toc["topic"] == "StructureAndOperation"][0]
end_page_index = first_page_index + 1
first_page, end_page = toc.loc[[first_page_index, end_page_index]]["page"].to_list()

In [6]:
from IPython.display import display
import duckdb

structure_type = "Not defined"

def get_sql_create_a_table_from_dataframe(df: pd.DataFrame, name: str) -> str:
    duckdb.sql(f"CREATE TABLE IF NOT EXISTS '{name}' AS SELECT * FROM df")

    # insert into the table "my_table" from the DataFrame "my_df"
    duckdb.sql(f"INSERT INTO '{name}' SELECT * FROM df")
    

for page in range(first_page, end_page):
    for table in doc[page].find_tables():
        df = table.to_pandas()
        if "Col1" in df.columns:
            columns_list = df.columns.to_list()
            columns_list[1:] = df.iloc[0].to_list()[1:]
            structure_type = columns_list[0]
            columns_list[0] = "detail"
            df.columns = columns_list
            df = df.iloc[1:]

        if "Col0" in df.columns:
            df.columns = columns_list
        get_sql_create_a_table_from_dataframe(df, structure_type)
        display(df)

,detail,2564,2565,2566
1,รายได้จากการดำเนินงานรวม (พันบาท),"163,348,945.00","173,257,605.00","192,653,621.00"
2,รายได้ดอกเบี้ยสุทธิ (พันบาท),"119,390,486.00","132,998,355.00","148,443,946.00"
3,รายได้ที่มิใช่ดอกเบี้ย (พันบาท),"43,958,459.00","40,259,250.00","44,209,675.00"


,detail,2564,2565,2566
0,รายได้จากการดำเนินงานรวม (%),100.00,100.00,100.00
1,รายได้ดอกเบี้ยสุทธิ (%),73.09,76.76,77.05
2,รายได้ที่มิใช่ดอกเบี้ย (%),26.91,23.24,22.95


,detail,2564,2565,2566
1,รายได้จากการดำเนินงานรวม (พันบาท),"163,348,945.00","173,257,605.00","192,653,621.00"
2,รายได้จากในประเทศ (พันบาท),"163,348,945.00","173,257,605.00","192,653,621.00"
3,รายได้จากต่างประเทศ (พันบาท),0.00,0.00,0.00


,detail,2564,2565,2566
0,รายได้จากการดำเนินงานรวม (%),100.00,100.00,100.00
1,รายได้จากในประเทศ (%),100.00,100.00,100.00
2,รายได้จากต่างประเทศ (%),0.00,0.00,0.00


,detail,2564,2565,2566
1,รายได้อื่นรวม (พันบาท),0.00,0.00,0.00
2,รายได้อื่นจากการดำเนินงาน (พันบาท),0.00,0.00,0.00
3,รายได้อื่นนอกเหนือจากการดำเนินงาน (พันบาท),0.00,0.00,0.00


,รายการ (หน่วย : พันบาท),2564,2565,2566
0,ค่าใช้จ่ายการทำวิจัยและพัฒนา (R&D) ในระยะเวลา ...,N/A,N/A,N/A


In [7]:
tables = duckdb.sql("SHOW TABLES")

In [8]:
tables.to_df()

,name
0,รายได้อื่น ตามที่ระบุในงบการเงิน
1,โครงสร้างรายได้ของสายผลิตภัณฑ์หรือกลุ่มธุรกิจ
2,โครงสร้างรายได้จากในประเทศและจากต่างประเทศ


In [30]:
table = "โครงสร้างรายได้จากในประเทศและจากต่างประเทศ"
print(duckdb.sql(f"SELECT * FROM '{table}'").to_df().drop_duplicates().to_markdown(index=False))

| detail                         | 2564           | 2565           | 2566           |
|:-------------------------------|:---------------|:---------------|:---------------|
| รายได้จากการดำเนินงานรวม (พันบาท) | 163,348,945.00 | 173,257,605.00 | 192,653,621.00 |
| รายได้จากในประเทศ (พันบาท)       | 163,348,945.00 | 173,257,605.00 | 192,653,621.00 |
| รายได้จากต่างประเทศ (พันบาท)      | 0.00           | 0.00           | 0.00           |
| รายได้จากการดำเนินงานรวม (%)     | 100.00         | 100.00         | 100.00         |
| รายได้จากในประเทศ (%)           | 100.00         | 100.00         | 100.00         |
| รายได้จากต่างประเทศ (%)          | 0.00           | 0.00           | 0.00           |


In [32]:
print(duckdb.sql("SELECT * FROM 'โครงสร้างรายได้ของสายผลิตภัณฑ์หรือกลุ่มธุรกิจ'").to_df().drop_duplicates().to_markdown(index=False))

| detail                         | 2564           | 2565           | 2566           |
|:-------------------------------|:---------------|:---------------|:---------------|
| รายได้จากการดำเนินงานรวม (พันบาท) | 163,348,945.00 | 173,257,605.00 | 192,653,621.00 |
| รายได้ดอกเบี้ยสุทธิ (พันบาท)         | 119,390,486.00 | 132,998,355.00 | 148,443,946.00 |
| รายได้ที่มิใช่ดอกเบี้ย (พันบาท)        | 43,958,459.00  | 40,259,250.00  | 44,209,675.00  |
| รายได้จากการดำเนินงานรวม (%)     | 100.00         | 100.00         | 100.00         |
| รายได้ดอกเบี้ยสุทธิ (%)             | 73.09          | 76.76          | 77.05          |
| รายได้ที่มิใช่ดอกเบี้ย (%)            | 26.91          | 23.24          | 22.95          |


In [34]:
print(duckdb.sql("SELECT * FROM 'รายได้อื่น ตามที่ระบุในงบการเงิน'").to_df().drop_duplicates().to_markdown(index=False))

| detail                                      | 2564   | 2565   | 2566   |
|:--------------------------------------------|:-------|:-------|:-------|
| รายได้อื่นรวม (พันบาท)                          | 0.00   | 0.00   | 0.00   |
| รายได้อื่นจากการดำเนินงาน (พันบาท)               | 0.00   | 0.00   | 0.00   |
| รายได้อื่นนอกเหนือจากการดำเนินงาน (พันบาท)        | 0.00   | 0.00   | 0.00   |
| ค่าใช้จ่ายการทำวิจัยและพัฒนา (R&D) ในระยะเวลา 3 ปี | N/A    | N/A    | N/A    |
| ที่ผ่านมา                                      |        |        |        |


In [27]:
print(doc[3].get_text().split("ประเภทธุรกิจ\n:")[1].strip().split("\nเลขทะเบียนบริษัท")[0])

ธนาคารกสิกรไทย จำกัด (มหาชน) ประกอบกิจการธนาคารพาณิชย์ ธุรกิจ
หลักทรัพย์และธุรกิจที่เกี่ยวเนื่องตามที่ได้รับอนุญาตไว้ในพระราชบัญญัติธุรกิจ
สถาบันการเงินฯ และพระราชบัญญัติหลักทรัพย์และตลาดหลักทรัพย์ฯ และ
ประกาศที่เกี่ยวข้อง โดย ณ วันที่ 31 ธันวาคม 2566 ธนาคารมีเครือข่ายสาขา
จำนวน 809 สาขา และเครื่องอิเล็กทรอนิกส์สำหรับทำธุรกรรมด้วยตนเอง
10,903 เครื่อง ครอบคลุมทุกพื้นที่และเพียงพอต่อความต้องการของลูกค้า
นอกจากนี้ ธนาคารมีเครือข่ายการให้บริการในต่างประเทศรวม 16 แห่ง ใน 8
ประเทศ มีศูนย์กลางการดำเนินงานและให้บริการที่สำนักงานใหญ่
